In [1]:
import pandas as pd

# Load the CSV file
csv_file = 'finalmpmb.csv'
df = pd.read_csv(csv_file)

# Drop all columns except the specified ones
columns_to_keep = ['Week Number', 'Seconds of Week', 'PRN', 'Amplitude']
df = df[columns_to_keep]
#df.to_csv("finalmpmb.csv", sep=',', index=False, encoding='utf-8')



# Print the first few rows
print("Head of the file:")
print(df.head())

# Print the last few rows
print("\nTail of the file:")
print(df.tail())


Head of the file:
   Week Number  Seconds of Week  PRN  Amplitude
0          267    215903.078125   24   0.040539
1          267    215903.078125   23   0.050577
2          267    215903.078125   29   0.052936
3          267    215903.078125   10   0.063523
4          267    215903.078125   18   0.064542

Tail of the file:
        Week Number  Seconds of Week  PRN  Amplitude
762792          267     302302.09375   10   0.061078
762793          267     302302.12500   27   0.043509
762794          267     302302.09375   23   0.047384
762795          267     302302.12500   29   0.038186
762796          267     302302.09375   15   0.034806


In [2]:
from datetime import datetime, timedelta

# Define the GPS epoch
GPS_EPOCH = datetime(1980, 1, 6)

def gps_to_utc(week, seconds):
    # Calculate the GPS time
    gps_time = GPS_EPOCH + timedelta(weeks=week, seconds=seconds)
    return gps_time

# Apply the conversion to the DataFrame
df['UTC Time'] = df.apply(lambda row: gps_to_utc(row['Week Number'], row['Seconds of Week']), axis=1)

# Convert 'UTC Time' to a 24-hour time format
df['24-hour Time'] = df['UTC Time'].dt.strftime('%H:%M:%S')

# Print the first few rows to verify
print("Head of the file with 24-hour time format:")
print(df.head())


Head of the file with 24-hour time format:
   Week Number  Seconds of Week  PRN  Amplitude                   UTC Time  \
0          267    215903.078125   24   0.040539 1985-02-19 11:58:23.078125   
1          267    215903.078125   23   0.050577 1985-02-19 11:58:23.078125   
2          267    215903.078125   29   0.052936 1985-02-19 11:58:23.078125   
3          267    215903.078125   10   0.063523 1985-02-19 11:58:23.078125   
4          267    215903.078125   18   0.064542 1985-02-19 11:58:23.078125   

  24-hour Time  
0     11:58:23  
1     11:58:23  
2     11:58:23  
3     11:58:23  
4     11:58:23  


In [3]:
# Round the "Seconds of Week" column to the nearest integer
#df['Seconds of Week'] = df['Seconds of Week'].round().astype(int)

# Normalize the "Seconds of Week" column by subtracting the first value
#first_seconds = df['Seconds of Week'].iloc[0]
#df['Seconds of Week'] = df['Seconds of Week'] - first_seconds

# Print the first few rows
#print("Head of the file:")
#print(df.head())

# Print the last few rows
#print("\nTail of the file:")
#print(df.tail())

In [3]:
import numpy as np
# Replace zero or very small values of "Amplitude" to avoid log10 issues
df['Amplitude'] = df['Amplitude'].replace(0, np.nan)
df['Amplitude'] = df['Amplitude'].fillna(df['Amplitude'].min() / 2)

# Add a new column 'd/u' which is 20log10(Amplitude)
df['d/u'] = -1*20 * np.log10(df['Amplitude'])

# Print the first few rows
print("Head of the file:")
print(df.head())

# Print the last few rows
print("\nTail of the file:")
print(df.tail())


Head of the file:
   Week Number  Seconds of Week  PRN  Amplitude                   UTC Time  \
0          267    215903.078125   24   0.040539 1985-02-19 11:58:23.078125   
1          267    215903.078125   23   0.050577 1985-02-19 11:58:23.078125   
2          267    215903.078125   29   0.052936 1985-02-19 11:58:23.078125   
3          267    215903.078125   10   0.063523 1985-02-19 11:58:23.078125   
4          267    215903.078125   18   0.064542 1985-02-19 11:58:23.078125   

  24-hour Time        d/u  
0     11:58:23  27.842479  
1     11:58:23  25.921019  
2     11:58:23  25.525008  
3     11:58:23  23.941381  
4     11:58:23  23.803174  

Tail of the file:
        Week Number  Seconds of Week  PRN  Amplitude  \
762792          267     302302.09375   10   0.061078   
762793          267     302302.12500   27   0.043509   
762794          267     302302.09375   23   0.047384   
762795          267     302302.12500   29   0.038186   
762796          267     302302.09375   15   0.

In [4]:
import plotly.io as pio
pio.renderers.default = "iframe"
# Ask user to input a specific PRN
import plotly.express as px
import plotly.graph_objects as go
# Plot d/u values against 24-hour time for all PRNs together
fig = go.Figure()

# Add traces for each PRN
for prn in df['PRN'].unique():
    group = df[df['PRN'] == prn]
    fig.add_trace(go.Scatter(x=group['24-hour Time'], y=group['d/u'], mode='lines', name=f'PRN {prn}'))

# Update layout
fig.update_layout(
    title='d/u values for all PRNs',
    xaxis_title='Time (24-hour format)',
    yaxis_title='d/u (dB)',
    xaxis_tickformat='%H:%M:%S'
)

# Show the figure
fig.show()


In [5]:
grouped = df.groupby('PRN')

# Print the head and tail of each group
for prn, group in grouped:
    print(f"\nPRN: {prn}")
    print("Head of the group:")
    print(group.head())
    print("\nTail of the group:")
    print(group.tail())


PRN: 2
Head of the group:
       Week Number  Seconds of Week  PRN  Amplitude  \
87754          267    227356.578125    2   0.076089   
87763          267    227357.578125    2   0.055402   
87772          267    227358.578125    2   0.080838   
87781          267    227359.578125    2   0.059998   
87790          267    227360.578125    2   0.060612   

                        UTC Time 24-hour Time        d/u  
87754 1985-02-19 15:09:16.578125     15:09:16  22.373546  
87763 1985-02-19 15:09:17.578125     15:09:17  25.129485  
87772 1985-02-19 15:09:18.578125     15:09:18  21.847665  
87781 1985-02-19 15:09:19.578125     15:09:19  24.437314  
87790 1985-02-19 15:09:20.578125     15:09:20  24.348814  

Tail of the group:
        Week Number  Seconds of Week  PRN  Amplitude  \
473812          267     269150.62500    2   0.105464   
473831          267     269152.65625    2   0.082157   
473841          267     269153.62500    2   0.130126   
473860          267     269155.62500    2   

In [6]:
%matplotlib widget
import matplotlib.pyplot as plt

# Ask user to input a specific PRN
specific_prn = int(input("Enter a PRN to plot (e.g., 32): "))

# Plot d/u values against 24-hour time for the specific PRN
if specific_prn in grouped.groups:
    group = grouped.get_group(specific_prn)
    plt.figure(figsize=(12, 6))
    plt.plot(group['24-hour Time'], group['d/u'], marker='o', linestyle='-', color='b')
    plt.title(f'd/u values for PRN {specific_prn}')
    plt.xlabel('Time (24-hour format)')
    plt.ylabel('d/u (dB)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()
else:
    print(f"PRN {specific_prn} not found in the dataset.")

Enter a PRN to plot (e.g., 32):  999


PRN 999 not found in the dataset.


In [7]:
import plotly.io as pio
pio.renderers.default = "iframe"


In [9]:
# Ask user to input a specific PRN
import plotly.express as px
import plotly.graph_objects as go
specific_prn = int(input("Enter a PRN to plot (e.g., 32): "))

# Plot d/u values against 24-hour time for the specific PRN
if specific_prn in grouped.groups:
    group = grouped.get_group(specific_prn)
    fig = px.line(group, x='24-hour Time', y='d/u', title=f'd/u values for PRN {specific_prn}')
    fig.update_layout(
        xaxis_title='Time (24-hour format)',
        yaxis_title='d/u (dB)',
        xaxis_tickformat='%H:%M:%S'
    )
    fig.show()
else:
    print(f"PRN {specific_prn} not found in the dataset.")


Enter a PRN to plot (e.g., 32):  99


PRN 99 not found in the dataset.


### 

In [10]:
# Count the number of values below 16
count_below_16 = df["d/u"][df["d/u"] < 16].count()

# Calculate the total number of values in the column
total_values = df["d/u"].count()

# Calculate the percentage of values below 16
percentage_below_16 = (count_below_16 / total_values) * 100

# Count the number of values below 20
count_below_20= df["d/u"][df["d/u"] < 20].count()

# Calculate the total number of values in the column
total_values = df["d/u"].count()

# Calculate the percentage of values below 16
percentage_below_20 = (count_below_20 / total_values) * 100

# Count the number of values >= 20
count_eqgt_20= df["d/u"][df["d/u"] >= 20].count()

# Calculate the total number of values in the column
total_values = df["d/u"].count()

# Calculate the percentage of values below 16
percentage_eqgt_20 = (count_eqgt_20 / total_values) * 100
print(f"Percentage of d/u values below 16: {percentage_below_16:.2f}%")
print(f"Percentage of d/u values below 20: {percentage_below_20:.2f}%")
print(f"Percentage of d/u values >= 20: {percentage_eqgt_20:.2f}%")




Percentage of d/u values below 16: 0.39%
Percentage of d/u values below 20: 3.65%
Percentage of d/u values >= 20: 96.35%
